In [2]:
import numpy as np
import pandas as pd 
import os

In [68]:
df = pd.read_csv(r'C:\Users\warag\OneDrive\Bureau\Transfers_Rumors\player_rumors.csv')
df.head()

,filename,date,player,rumors
0,4GuMWHaJh_g.json,2024-10-05T12:56:09,Rudiger,[Free transfer]
1,4GuMWHaJh_g.json,2024-10-05T12:56:09,Juan Martinez,[Transfer to Real Madrid]
2,4GuMWHaJh_g.json,2024-10-05T12:56:09,Inter Milan,Inter Milan has been mentioned as a potential ...
3,4GuMWHaJh_g.json,2024-10-05T12:56:09,Paul Pogba,[Set to return to professional football]
4,4GuMWHaJh_g.json,2024-10-05T12:56:09,Ferland Mendy Mende,Real Madrid has set to extend the contract of ...


In [ ]:
df['rumors'] = df['rumors'].str.strip('[]')
df['date'] = pd.to_datetime(df['date']).dt.strftime('%Y/%m/%d')
df = df[df['player'] != "Sala"]


In [76]:
pd.set_option('display.max_rows', None)

rename_map = {
    'Nico': 'Nico williams',
    'Merino': 'Mikel Merino'}
df['player'] = df['player'].replace(rename_map)
counts_df = df['player'].value_counts().reset_index()
counts_df.columns = ['player', 'count']
print(counts_df)


                                                player  count
0                                          Real Madrid     24
1                                       Victor Osimhen     16
2                               Trent Alexander-Arnold     13
3                                                Milan     12
4                                         Jadon Sancho     12
5                                      Conor Gallagher      9
6                                             Juventus      9
7                                        Dani Carvajal      9
8                                        Nico williams      9
9                                      Fabrizio Romano      8
10                                       Romelu Lukaku      8
11                                        Ruben Amorim      8
12                                         Aston Villa      8
13                                     Virgil van Dijk      8
14                                         Inter Milan      7
15      

,filename,date,player,rumors
205,a9AG-k7oQ10.json,2024/07/21,Nico williams,FC Barcelona is focused on signing him and the...
289,CkjWzY0C9ug.json,2024/07/31,Nico williams,FC Barcelona are giving total priority to the ...
304,cLealaIts8Q.json,2025/02/01,Nico williams,"is on Manchester City, Peter Reisch, Omar Marm..."
333,Cr2rfAK54yc.json,2024/07/28,Nico williams,There are reports that Barcelona is waiting fo...
393,d67dD7GxIW0.json,2024/08/20,Nico williams,There is no update on Nico's transfer situatio...
420,DYauRksn52g.json,2024/08/01,Nico williams,"Barcelona is considering signing him, but no d..."
595,JNNxpoejkmQ.json,2024/07/23,Nico williams,"rumored to be a target for Barcelona, with neg..."
856,VfwK9VV46ww.json,2024/12/29,Nico williams,reportedly being considered for a move to Ital...
931,zc71Mwj1BSQ.json,2025/02/06,Nico williams,Real Madridly has expressed interest in signin...


In [50]:
df

,filename,date,player,rumors
0,4GuMWHaJh_g.json,2024/10/05,Rudiger,Free transfer
1,4GuMWHaJh_g.json,2024/10/05,Juan Martinez,Transfer to Real Madrid
3,4GuMWHaJh_g.json,2024/10/05,Paul Pogba,Set to return to professional football
4,4GuMWHaJh_g.json,2024/10/05,Ferland Mendy Mende,Real Madrid has set to extend the contract of ...
5,4GuMWHaJh_g.json,2024/10/05,Antonio Rüdiger,Tony Rudiger is on the market and available as...
...,...,...,...,...
939,ZF8azXGgSvQ.json,2025/01/10,Marcus Rashford,Manchester United have been linked to a move f...
947,_-MFPoP5tIQ.json,2025/01/01,Trent Alexander-Arnold,Liverpool are reportedly close to agreeing a d...
948,_-MFPoP5tIQ.json,2025/01/01,Nicopas,"Chelsea have signed Nikola Nikolovic, also kno..."
949,_-MFPoP5tIQ.json,2025/01/01,Anthony,Manchester United are reportedly interested in...


In [78]:
df.to_csv('rumors.csv')